In [27]:
# Import modules and subroutines:

include("../subroutines/Subroutines.jl");

In [2]:
# Load the chemical data into julia as an array of chemical data structs:
# (one struct obtained for each molecular geometry as set up in the config file)

cdata_list = ReadIn("../datasets/pyscf_data/h2o_sto-3g_080222%000756.hdf5");
chemical_data = cdata_list[1]
PrintChemData(chemical_data)

Molecule name: h2o
Basis set: sto-3g
Molecular geometry: O 0.0 0.0 0.0; H 1.0 0.0 0.0; H 0.0 1.0 0.0; 
RHF energy: -74.96117113786774
FCI energy: -75.01927599660624


In [3]:
# Run a DMRG calculation (i.e. a single-state subspace):

dmrg1 = GenSubspace(chemical_data, 1, psi_maxdim=7, sweep_num=10, sweep_noise=(1e-6, 1e-7, 1e-8, 0.0));
dmrg1.ord_list = [collect(1:dmrg1.chem_data.N_spt)]
GenStates!(dmrg1, verbose=true);

bipartitions, entropies = ComputeBipartites(dmrg1, ns=40)
println(length(bipartitions))

Generating states:
Progress: [1/1] 
Done!
63


In [10]:
tnqe1 = GenSubspace(chemical_data, 6, psi_maxdim=3, sweep_num=6, sweep_noise=(1e-6), thresh="projection", eps=1e-12);

# Initialize with random orderings:

tnqe1.ord_list = [randperm(tnqe1.chem_data.N_spt) for j=1:tnqe1.mparams.M]

6-element Vector{Vector{Int64}}:
 [3, 5, 6, 2, 1, 4, 7]
 [2, 6, 4, 3, 5, 7, 1]
 [4, 5, 1, 3, 2, 7, 6]
 [5, 3, 1, 7, 4, 6, 2]
 [3, 2, 4, 7, 1, 5, 6]
 [4, 7, 3, 6, 5, 1, 2]

In [22]:
# Sequentially optimized orderings:

tnqe1.ord_list = []

for k=1:tnqe1.mparams.M
    
    tnqe1.ord_list = vcat(tnqe1.ord_list, [randperm(tnqe1.chem_data.N_spt)])
    
    tnqe1.ord_list, statevec = BipartiteAnnealing(
        tnqe1.ord_list, 
        bipartitions, 
        entropies, 
        tnqe1.mparams.psi_maxdim,
        anchor=true,
        anum=k-1,
        maxiter=2000, 
        swap_mult=0.5, 
        alpha=1e-5, 
        stun=true, 
        gamma=1e4, 
        zeta=0.03,
        xi=0.3,
        opt_statevec=true,
        delta=1e-8,
        verbose=true
    );
    
    println("")
    
end

tnqe1.ord_list, statevec = BipartiteAnnealing(
        tnqe1.ord_list, 
        bipartitions, 
        entropies, 
        tnqe1.mparams.psi_maxdim,
        anchor=true,
        anum=1,
        maxiter=2000, 
        swap_mult=0.5, 
        alpha=1e-5, 
        stun=true, 
        gamma=1e4, 
        zeta=0.03,
        xi=0.3,
        opt_statevec=true,
        delta=1e-8,
        verbose=true
    );


0.5591038337365366    
0.37343085185138575    
0.26478347182461814    
0.197680411536657      
0.14964645429499426    
0.11423555757342337    


In [23]:
# Generate the permutation operators:

GenPermOps!(tnqe1, verbose=true)

Generating permutation operators:
Progress: [15/15] 
Done!



In [41]:
# Initialize the states:
opt_sweeps = Sweeps(15)
maxdim!(opt_sweeps,tnqe1.mparams.psi_maxdim)
mindim!(opt_sweeps,tnqe1.mparams.psi_maxdim)
cutoff!(opt_sweeps,1e-12)
setnoise!(opt_sweeps, 1e-1,1e-1,1e-1,1e-2,1e-3,1e-4,1e-5,1e-6,1e-8,1e-10,1e-12,0.0)

#GenStates!(tnqe1, sweeps=opt_sweeps, verbose=true);
#levs = reverse(collect(1:tnqe1.mparams.M))
#levs = randperm(tnqe1.mparams.M)
GenExcited!(tnqe1, sweeps=opt_sweeps, weight = 2.0, levs=levs, verbose=true)
tnqe1.rev_list = [false for j=1:tnqe1.mparams.M]
SelectPermOps!(tnqe1)

GenSubspaceMats!(tnqe1, verbose=true)

SolveGenEig!(tnqe1, thresh="inversion", eps=1e-8, verbose=false)

Generating states:
Progress: [3/3] 
Done!
Generating states:
Progress: [4/4] 
Done!
Generating states:
Progress: [1/1] 
Done!
Generating states:
Progress: [6/6] 
Done!
Generating states:
Progress: [5/5] 
Done!
Generating states:
Progress: [2/2] 
Done!


In [42]:
# Flip-optimization:

N_flip = Int(floor(tnqe1.mparams.M/2))

println("")
println("FLIP-OPTIMIZATION:")
println("")

for k=1:16
    
    MultiGeomOptim!(
        tnqe1, 
        sweeps=1, 
        method="geneig",
        maxiter=8000,
        noise=[10.0^(-3)],
        thresh="inversion",
        eps=1e-8,
        restrict_svals=true,
        costfunc="energy",
        theta=[1.0,0.0,0.0],
        verbose=true
    )
    
    println("")
    
    j_list = shuffle(collect(1:tnqe1.mparams.M))[1:N_flip]
    ReverseStates!(tnqe1, j_list, verbose=true)
    
end



FLIP-OPTIMIZATION:

Sweep: [1/1]; site: [6/6]; E_min = -83.84840508006947; kappa = 18.27143751523939              
Reversing states...
Done!
Sweep: [1/1]; site: [6/6]; E_min = -83.85259829776207; kappa = 17.43525579375933              
Reversing states...
Done!
Sweep: [1/1]; site: [6/6]; E_min = -83.85436782375156; kappa = 9.936590168638562              
Reversing states...
Done!
Sweep: [1/1]; site: [6/6]; E_min = -83.85713012659956; kappa = 37.8102586801809               
Reversing states...
Done!
Sweep: [1/1]; site: [6/6]; E_min = -83.85759232147637; kappa = 64.25319979205881              
Reversing states...
Done!
Sweep: [1/1]; site: [6/6]; E_min = -83.85777179970526; kappa = 116.20497815992454             
Reversing states...
Done!
Sweep: [1/1]; site: [6/6]; E_min = -83.8579305842304; kappa = 117.01485000016245              
Reversing states...
Done!
Sweep: [1/1]; site: [6/6]; E_min = -83.85786396726111; kappa = 237.7615521175678              
Reversing states...
Done!
Sweep: [1/1

In [43]:
# Fine optimization:

println("")
println("FINE OPTIMIZATION:")
println("")

for k=4:6
    
    MultiGeomOptim!(
        tnqe1, 
        sweeps=1, 
        method="geneig",
        noise=[10.0^(-k)],
        thresh="inversion",
        eps=1e-6,
        restrict_svals=false,
        verbose=true
    )

    println("")
    j_list = shuffle(collect(1:tnqe1.mparams.M))[1:N_flip]
    #j_list = collect(1:tnqe1.mparams.M)
    ReverseStates!(tnqe1, j_list, verbose=true)
    println("")
    
    MultiGeomOptim!(
        tnqe1, 
        sweeps=1, 
        method="geneig",
        noise=[1e-10],
        thresh="inversion",
        eps=1e-6,
        restrict_svals=true,
        verbose=true
    )

    println("")
    j_list = shuffle(collect(1:tnqe1.mparams.M))[1:N_flip]
    #j_list = collect(1:tnqe1.mparams.M)
    ReverseStates!(tnqe1, j_list, verbose=true)
    println("")
    
end

println("")
println("FINISHING TOUCHES:")
println("")

finishing_noise = [10.0^(-8),10.0^(-10),0.0,0.0,0,0,0,0]

for k=1:6

    MultiGeomOptim!(
            tnqe1, 
            sweeps=1, 
            method="geneig",
            noise=[finishing_noise[k]],
            thresh="inversion",
            eps=1e-6,
            restrict_svals=true,
            verbose=true
        )
    
    println("")
    j_list = shuffle(collect(1:tnqe1.mparams.M))[1:N_flip]
    #j_list = collect(1:tnqe1.mparams.M)
    ReverseStates!(tnqe1, j_list, verbose=true)
    println("")
    
end


FINE OPTIMIZATION:

Sweep: [1/1]; site: [6/6]; E_min = -83.85942169822312; kappa = 47.34264814546805              
Reversing states...
Done!

Sweep: [1/1]; site: [6/6]; E_min = -83.85954423914248; kappa = 30.358811478816335             
Reversing states...
Done!

Sweep: [1/1]; site: [6/6]; E_min = -83.85973781784996; kappa = 29.910652755861673             
Reversing states...
Done!

Sweep: [1/1]; site: [6/6]; E_min = -83.85977590981615; kappa = 36.486399835148696             
Reversing states...
Done!

Sweep: [1/1]; site: [6/6]; E_min = -83.85978854490605; kappa = 39.076723091073056             
Reversing states...
Done!

Sweep: [1/1]; site: [6/6]; E_min = -83.8598049123135; kappa = 36.23566836728724               
Reversing states...
Done!


FINISHING TOUCHES:

Sweep: [1/1]; site: [6/6]; E_min = -83.85981169131456; kappa = 34.965118914317955             
Reversing states...
Done!

Sweep: [1/1]; site: [6/6]; E_min = -83.85981485821516; kappa = 33.453020736240894             
Reversing